In [1]:
import importlib
import numpy as np
import pandas as pd
import utils

In [2]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/stacked_lstm/utils.py'>

In [3]:
from keras.models import Sequential, load_model
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten, Conv1D, AveragePooling1D, Bidirectional
# from keras.optimizers import Adam

Using TensorFlow backend.


In [4]:
emb = utils.load_embedding()

In [5]:
emb.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.536122,0.184517,0.187153,1.107727,-0.072044,-0.394318,0.180612,-0.317482,-0.387441,-0.005007,...,-0.096253,0.181341,-0.214352,0.24542,0.866946,0.628616,0.314401,-0.128581,-0.317977,0.842992
1,-0.584020,0.390310,0.652820,-0.340300,0.194930,-0.834890,0.119290,-0.572910,-0.568440,0.729890,...,0.285860,-0.052162,-0.508180,-0.63459,0.338890,0.284160,-0.203400,-1.233800,0.467150,0.788580
2,0.257690,0.456290,-0.769740,-0.376790,0.592720,-0.063527,0.205450,-0.573850,-0.290090,-0.136620,...,0.030498,-0.395430,-0.385150,-1.00020,0.087599,-0.310090,-0.346770,-0.314380,0.750040,0.970650
3,-0.039369,1.203600,0.354010,-0.559990,-0.520780,-0.669880,-0.754170,-0.653400,-0.232460,0.586860,...,-0.601410,0.504030,-0.083316,0.20239,0.443000,-0.060769,-0.428070,-0.084135,0.491640,0.085654
4,-0.268200,-0.478700,0.180990,-0.538370,-0.240210,-0.562030,0.209440,0.423580,-0.461470,0.769030,...,0.005399,-0.533440,-0.187060,0.52254,0.243610,0.051387,0.272100,-0.444330,0.019464,0.627820


In [6]:
emb_dim = emb.shape[1]

In [7]:
emb_dim

50

In [4]:
word2ind, ind2word = utils.load_index_word_map()

In [5]:
vocab_size = len(word2ind)

In [6]:
dataset = utils.load_training_samples()

In [7]:
train_x, val_x, train_y, val_y = utils.train_valid_split(dataset.values[:, :-1], dataset.values[:, -1:], test_size=0, random_state=43)

In [8]:
train_x.shape

(2090316, 200)

In [9]:
train_y.shape

(2090316, 1)

In [10]:
val_x.shape

(0, 200)

In [11]:
val_y.shape

(0, 1)

In [16]:
input_length = train_x.shape[1]

In [12]:
model = load_model('model20180324.h5')

In [18]:
emb = model.get_weights()[0]

In [19]:
emb_dim = emb.shape[1]

In [20]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_length, weights=[emb]))
model.add(Conv1D(filters=256, kernel_size=8, padding='same', activation='relu'))
model.add(AveragePooling1D(pool_size=2, padding='same'))
model.add(Dropout(.2))
model.add(Conv1D(filters=32, kernel_size=8, padding='same', activation='relu'))
model.add(AveragePooling1D(pool_size=2, padding='same'))
model.add(Dropout(0.2))
model.add(LSTM(50, dropout=.2, recurrent_dropout=.2))
model.add(Dropout(.2))
model.add(Dense(vocab_size, activation='softmax'))

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 200, 50)           2668750   
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 200, 256)          102656    
_________________________________________________________________
average_pooling1d_7 (Average (None, 100, 256)          0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 100, 256)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 100, 32)           65568     
_________________________________________________________________
average_pooling1d_8 (Average (None, 50, 32)            0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 50, 32)            0         
__________

In [13]:
model.get_layer('embedding_4').trainable = False

In [14]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 200, 50)           2668750   
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 200, 256)          102656    
_________________________________________________________________
average_pooling1d_7 (Average (None, 100, 256)          0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 100, 256)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 100, 32)           65568     
_________________________________________________________________
average_pooling1d_8 (Average (None, 50, 32)            0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 50, 32)            0         
__________

In [29]:
model.fit(train_x, train_y, validation_data=[val_x, val_y], epochs=1, batch_size=1000, shuffle=True)

Train on 5880677 samples, validate on 309509 samples
Epoch 1/1
5880677/5880677 [==============================] - 1302s - loss: 5.0200 - acc: 0.2017 - val_loss: 4.4740 - val_acc: 0.2557


In [30]:
# save model
model.save('model20180324.h5')

In [18]:
model.fit(train_x, train_y, epochs=1, batch_size=2500, shuffle=True)

Epoch 1/1
2090316/2090316 [==============================] - 361s - loss: 4.6452 - acc: 0.2365   


In [19]:
# save model
model.save('model20180324.h5')

In [23]:
model.fit(train_x, train_y, epochs=5, batch_size=2000, shuffle=True)

Epoch 1/5
6190186/6190186 [==============================] - 1183s - loss: 4.3361 - acc: 0.2677  
Epoch 2/5
6190186/6190186 [==============================] - 1192s - loss: 4.2954 - acc: 0.2707  
Epoch 3/5
6190186/6190186 [==============================] - 1220s - loss: 4.2649 - acc: 0.2734  
Epoch 4/5
6190186/6190186 [==============================] - 1216s - loss: 4.2408 - acc: 0.2754  
Epoch 5/5
6190186/6190186 [==============================] - 1223s - loss: 4.2206 - acc: 0.2771  


In [24]:
# save model
model.save('model20180324.h5')

In [29]:
model.fit(train_x, train_y, epochs=5, batch_size=1000, shuffle=True)

Epoch 1/5
6190186/6190186 [==============================] - 1701s - loss: 13.8982 - acc: 0.1381  
Epoch 2/5
6190186/6190186 [==============================] - 1700s - loss: 13.8982 - acc: 0.1381  
Epoch 3/5
6190186/6190186 [==============================] - 1704s - loss: 13.8982 - acc: 0.1381  
Epoch 4/5
6190186/6190186 [==============================] - 1708s - loss: 13.8982 - acc: 0.1381  
Epoch 5/5
6190186/6190186 [==============================] - 1710s - loss: 13.8982 - acc: 0.1381  


In [30]:
# save model
model.save('model20180324.h5')